### Import Required Libraries and Set Up Environment Variables

In [21]:
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime

# Manually set API key (since .env may not be used)
NASA_API_KEY = "ryu9cPgIYCiC2wrhZJDBydoiCNVWPziJWeuqk51y"

# Confirm API Key is loaded
print(f"NASA API Key Loaded: {NASA_API_KEY is not None}")

# Define NASA API Key
NASA_API_KEY = "ryu9cPgIYCiC2wrhZJDBydoiCNVWPziJWeuqk51y"  # Use your actual API key

# Confirm API Key is loaded
print(f"NASA API Key Loaded: {NASA_API_KEY is not None}")




NASA API Key Loaded: True
NASA API Key Loaded: True


### CME Data

In [3]:
# Define the API endpoint
base_url = "https://api.nasa.gov/DONKI/"
specifier = "CME"
startDate = "2013-05-01"
endDate = "2024-05-01"

# Build the query URL
query_url_CME = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
response = requests.get(query_url_CME)

In [5]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = response.json()

In [6]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:2], indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [7]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

In [8]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [9]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []


# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID = cme_df.loc[i, 'activityID']
    startTime = cme_df.loc[i, 'startTime']
    linkedEvents = cme_df.loc[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    for item in linkedEvents:
        expanded_rows.append({'cmeID': activityID, 'startTime_CME': startTime, 'linkedEvents': item})
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows
cme_expanded_df = pd.DataFrame(expanded_rows)
cme_expanded_df.head()

,cmeID,startTime_CME,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [10]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict.get('activityID', None)
    except (ValueError, TypeError):
        return None
        # Log the error or print it for debugging




In [11]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_expanded_df['GST_ActivityID'] = cme_expanded_df['linkedEvents'].apply(extract_activityID_from_dict)


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_expanded_df = cme_expanded_df.dropna(subset=['GST_ActivityID'])

In [13]:
# print out the datatype of each column in this DataFrame:
cme_expanded_df.head()

,cmeID,startTime_CME,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [14]:
# Convert the 'GST_ActivityID' column to string format 
cme_expanded_df['GST_ActivityID'] = cme_expanded_df['GST_ActivityID'].astype(str)
# Convert startTime to datetime format  
cme_expanded_df['startTime_CME'] = pd.to_datetime(cme_expanded_df['startTime_CME'])
# Rename startTime to startTime_CME and activityID to cmeID
cme_expanded_df = cme_expanded_df[cme_expanded_df['GST_ActivityID'].str.contains('GST')]

# Drop linkedEvents
cme_expanded_df = cme_expanded_df.drop(columns=['linkedEvents'])
# Verify that all steps were executed correctly
cme_expanded_df.head()

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


In [14]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  


,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [22]:
# Define API URL for GST data
specifier = "GST"

# Build the query URL for Geomagnetic Storms (Correcting API Key)
query_url_GST = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"




In [23]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)

In [24]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[:2], indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [27]:
import json

# Print the first few results to inspect the API response structure
print(json.dumps(gst_json[:2], indent=4))


[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [39]:
# Re-fetch GST data from API
gst_response = requests.get(query_url_GST)
gst_json = gst_response.json()

# Convert JSON to DataFrame
gst_df = pd.DataFrame(gst_json)

# Print available columns
print("Available columns:", gst_df.columns)

Available columns: Index(['gstID', 'startTime', 'allKpIndex', 'link', 'linkedEvents',
       'submissionTime', 'versionId'],
      dtype='object')


In [38]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df = gst_df.dropna(subset=['linkedEvents'])

gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [30]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [31]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

In [33]:
print("Available columns:", gst_df.columns)


Available columns: Index(['gstID', 'startTime', 'linkedEvents'], dtype='object')


In [41]:
# Check the first few rows of 'linkedEvents' before extraction
print(gst_df[['linkedEvents']].head())


                                    linkedEvents
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}
1  {'activityID': '2013-06-02T20:24:00-CME-001'}
2                                           None
3  {'activityID': '2013-09-29T22:40:00-CME-001'}
4  {'activityID': '2013-10-02T01:54:00-IPS-001'}


In [40]:
# Ensure linkedEvents is expanded
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

# Define function to extract 'activityID'
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict.get('activityID', None)
    except (AttributeError, TypeError):
        return None

# Apply function to extract CME_ActivityID
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(extract_activityID_from_dict)

# Print first few rows to verify
print(gst_df[['gstID', 'CME_ActivityID']].head())


                         gstID               CME_ActivityID
0  2013-06-01T01:00:00-GST-001  2013-05-31T15:45:00-HSS-001
1  2013-06-07T03:00:00-GST-001  2013-06-02T20:24:00-CME-001
2  2013-06-29T03:00:00-GST-001                         None
3  2013-10-02T03:00:00-GST-001  2013-09-29T22:40:00-CME-001
4  2013-10-02T03:00:00-GST-001  2013-10-02T01:54:00-IPS-001


In [42]:
print("Available columns after extraction:", gst_df.columns)


Available columns after extraction: Index(['gstID', 'startTime', 'allKpIndex', 'link', 'linkedEvents',
       'submissionTime', 'versionId', 'CME_ActivityID'],
      dtype='object')


In [44]:
# Check if gst_df exists
try:
    print(gst_df.shape)  # Check the DataFrame dimensions
except NameError:
    print("Error: gst_df is not defined. Re-run the previous cells to recreate it.")


(217, 8)


In [45]:
# Convert the 'CME_ActivityID' column to string format 

# Convert the 'gstID' column to string format 

# Convert startTime to datetime format  
gst_df['startTime_GST'] = pd.to_datetime(gst_df['startTime'])
gst_df = gst_df.drop(columns=['linkedEvents', 'startTime'])
# Rename startTime to startTime_GST 
gst_df.rename(columns={'activityID': 'gstID'}, inplace=True)
# Drop linkedEvents

# Verify that all steps were executed correctly
gst_df.head()

,gstID,allKpIndex,link,submissionTime,versionId,CME_ActivityID,startTime_GST
0,2013-06-01T01:00:00-GST-001,"[{'observedTime': '2013-06-01T01:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,2013-07-15T19:26Z,1,2013-05-31T15:45:00-HSS-001,2013-06-01 01:00:00+00:00
1,2013-06-07T03:00:00-GST-001,"[{'observedTime': '2013-06-07T03:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,2013-07-15T19:41Z,1,2013-06-02T20:24:00-CME-001,2013-06-07 03:00:00+00:00
2,2013-06-29T03:00:00-GST-001,"[{'observedTime': '2013-06-29T03:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,2013-09-25T04:48Z,2,None,2013-06-29 03:00:00+00:00
3,2013-10-02T03:00:00-GST-001,"[{'observedTime': '2013-10-02T06:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,2013-10-02T13:23Z,1,2013-09-29T22:40:00-CME-001,2013-10-02 03:00:00+00:00
4,2013-10-02T03:00:00-GST-001,"[{'observedTime': '2013-10-02T06:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,2013-10-02T13:23Z,1,2013-10-02T01:54:00-IPS-001,2013-10-02 03:00:00+00:00


In [23]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  


,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [46]:
# Check if gst_df exists
try:
    print(gst_df.shape)  # Print DataFrame dimensions
except NameError:
    print("Error: gst_df is not defined. Re-run the previous cells to recreate it.")


(217, 7)


In [47]:
try:
    print(cme_expanded_df.shape)
except NameError:
    print("Error: cme_expanded_df is not defined. Re-run the CME processing cells.")


(61, 3)


In [48]:
# Fetch GST data again
gst_response = requests.get(query_url_GST)
gst_json = gst_response.json()

# Convert JSON response to DataFrame
gst_df = pd.DataFrame(gst_json)

# Keep relevant columns
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df = gst_df.dropna(subset=['linkedEvents'])

# Expand linkedEvents column
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

# Extract CME_ActivityID from linkedEvents
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict.get('activityID', None)
    except (AttributeError, TypeError):
        return None

gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(extract_activityID_from_dict)

# Drop missing CME_ActivityID
gst_df = gst_df.dropna(subset=['CME_ActivityID'])

# Convert columns to correct formats
gst_df['startTime_GST'] = pd.to_datetime(gst_df['startTime'])
gst_df = gst_df.drop(columns=['linkedEvents', 'startTime'])


In [49]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    gst_df, 
    cme_expanded_df, 
    left_on=['gstID', 'CME_ActivityID'], 
    right_on=['GST_ActivityID', 'cmeID'], 
    how='inner'
)
# Drop duplicate column
merged_df = merged_df.drop(columns=['GST_ActivityID'])

# Preview merged DataFrame
print("Merged DataFrame Preview:")
merged_df.head()

Merged DataFrame Preview:


,gstID,CME_ActivityID,startTime_GST,cmeID,startTime_CME
0,2013-06-07T03:00:00-GST-001,2013-06-02T20:24:00-CME-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00
1,2013-10-02T03:00:00-GST-001,2013-09-29T22:40:00-CME-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00
2,2013-12-08T00:00:00-GST-001,2013-12-04T23:12:00-CME-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00
3,2014-02-19T03:00:00-GST-001,2014-02-16T14:15:00-CME-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00
4,2014-02-20T03:00:00-GST-001,2014-02-18T01:25:00-CME-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00


In [25]:
# Verify that the new DataFrame has the same number of rows as cme and gst


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           67 non-null     string             
 1   startTime_GST   67 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  67 non-null     object             
 3   cmeID           67 non-null     object             
 4   startTime_CME   67 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  67 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.3+ KB


### Computing the time it takes for a CME to cause a GST

In [52]:
# Compute time difference between CME and GST
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

# Preview the first few rows
print("Time Difference Calculated:")
merged_df[['gstID', 'CME_ActivityID', 'timeDiff']].head()


Time Difference Calculated:


,gstID,CME_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-02T20:24:00-CME-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-09-29T22:40:00-CME-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-04T23:12:00-CME-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-16T14:15:00-CME-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-18T01:25:00-CME-001,2 days 01:35:00


In [27]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


,timeDiff
count,67
mean,2 days 21:35:13.432835820
std,1 days 00:02:46.681279427
min,1 days 05:36:00
25%,2 days 03:12:00
50%,2 days 17:48:00
75%,3 days 12:17:00
max,6 days 03:00:00


### Exporting data in csv format

In [53]:
# Export data to CSV without the index
merged_df.to_csv("final_dataset.csv", index=False)

print("✅ Final dataset saved as 'final_dataset.csv'")

✅ Final dataset saved as 'final_dataset.csv'
